<h1>m2w1_JSON_weather_watkins_challenge_1122</h2>

<b>Challenge Steps</b>
<ol>
<li>New Project</li>
<li>New Venv</li>
<li>New Jupyter</li>
<li>Download JSON file "nws_weather_4HourForecast.json" to the root directory of your python project.</li>
<li>Import the json file into a Python dictionary.</li>
<li>Iterate through the "Properties" key, loading keys and values into a Pandas dataframe</li>
<li>Display as a table the following columns:
"startTime" ,"shortForecast" ,"temperature" ,"probabilityOfPrecipitation" ,"relativeHumidity"
</li>
</ol>
 


In [726]:
import json
import pandas as pd
import requests
import pytz
from datetime import datetime, timedelta

In [606]:
# open data in raw format in read only mode as variable called file
with open(r'D:\00 Development\Code KY\challenges\m2w1_JSON_weather_watkins_challenge_1122\nws_weather_4HourForecast.json', 'r') as file:
    data = json.load(file) # Load JSON content from the file into a Python dictionary

# Extract 'periods' data from JSON, convert to DataFrame, and assign to weather_info
weather_info = pd.json_normalize(data["properties"]["periods"])
# Extract specific columns from weather_info and assign to selected_weather_info
selected_weather_info = weather_info[["startTime", "shortForecast", "temperature", "probabilityOfPrecipitation.value", "relativeHumidity.value"]]
selected_weather_info



,startTime,shortForecast,temperature,probabilityOfPrecipitation.value,relativeHumidity.value
0,2024-11-16T11:00:00-05:00,Partly Sunny,51,0,71
1,2024-11-16T12:00:00-05:00,Partly Sunny,53,0,69
2,2024-11-16T13:00:00-05:00,Partly Sunny,54,0,69
3,2024-11-16T14:00:00-05:00,Partly Sunny,54,0,66


<b>More Challenge</b>

The above not enough for you? Try these:
<ol>
<li>download the JSON file directly into your Jupyter notebook by making a URL request to the address shown above</li>
<li>rename columns to shorter, easier-to-display names</li>
<li>reformat the datetime value to something easier to read for someone in its offset timezone (US Eastern)</li>
<li>convert the dewpoint's Celsius value to Fahrenheit</li>
<li>concatenate columns "temperature" and "temperatureUnit" so that the reader better understands the "temperature" value</li>
<li>use a json datafile that you download directly from the source "https://api.weather.gov"</li>
</ol>


In [773]:
# Set variable to URL for resource
url = 'https://raw.githubusercontent.com/stevewatkins17/PythonForDataAnalysis/refs/heads/mainline/Data/nws_weather_4HourForecast.json'
response = requests.get(url) # set response to data from URL

if response.status_code == 200:  # if response is successful
    # Convert response data to a dictionary, extract 'periods' section, and convert to DataFrame
    data_1 = pd.json_normalize(response.json()["properties"]["periods"])
else:  # If json download fails
    print(response.status_code)

# Sets the variable selected... to specific selected colums from data_1 and renames them.
selected_weather_data = data_1[["startTime","dewpoint.unitCode","dewpoint.value", "shortForecast", "temperature","temperatureUnit", "probabilityOfPrecipitation.value", "relativeHumidity.value"]].rename(columns={ "startTime": "Start","dewpoint.unitCode":"unit","dewpoint.value":"Dewpoint","shortForecast": "Forecast", "temperature": "Temp", "temperatureUnit": "T Unit", "probabilityOfPrecipitation.value": "Precipitation", "relativeHumidity.value": "Humidity"})

#--- Reformat start date time value and set eastern timezone ---
selected_weather_data_1 = selected_weather_data

def utc_to_est(row):

    # Extract the timezone part (-05:00) to a variable 
    timezone_offset = row['Start'][-6:]

    # Remove timezone from time
    row['Start'] = row['Start'][:-6]

    # convert Start string to datetime
    row['Start'] = pd.to_datetime(row['Start'])

    # Set Date time format to UTC initially
    row['Start'] = row['Start'].tz_localize('UTC', ambiguous='NaT')

    # Adjust to Eastern time
    row['Start'] = row['Start'].tz_convert('US/Eastern')

    # Format date time
    row['Start'] = row['Start'].strftime('%y/%m/%d %H:%M %Z')

    return row

selected_weather_data_1 = selected_weather_data_1.apply(utc_to_est, axis=1)

#--- Convert Dewpoints Celcius value to F change the value to str in order to include Deg F in cell with temp for dewpoint ---
def cel_to_f(row):
    if row['unit'] == 'wmoUnit:degC':
        row['Dewpoint'] = str((row['Dewpoint'] * 9/5) + 32) + ' \u00B0F' 
        row['unit'] = 'deg: F'
    return row

# Modify any row that has the unit celcius by using the .apply method running the cel_to_f function
selected_weather_data_1 = selected_weather_data_1.apply(cel_to_f, axis=1)

#-- Concatinate temp and temp unit --
# function to iterate over the rows and concat each
def concat_temp(row):

    # set temp in each row to string adding the degree symbol and the value of the unit row
    row['Temp'] = str(row['Temp']) + ' \u00B0' + str(row['T Unit'])
    return row

# call function and apply it to each row
selected_weather_data_1 = selected_weather_data_1.apply(concat_temp, axis=1)
    
selected_weather_data_1[['Start', 'Dewpoint', 'Forecast', 'Temp', 'Precipitation', 'Humidity']]


,Start,Dewpoint,Forecast,Temp,Precipitation,Humidity
0,24/11/16 06:00 EST,42.0 °F,Partly Sunny,51 °F,0,71
1,24/11/16 07:00 EST,43.0 °F,Partly Sunny,53 °F,0,69
2,24/11/16 08:00 EST,44.0 °F,Partly Sunny,54 °F,0,69
3,24/11/16 09:00 EST,43.0 °F,Partly Sunny,54 °F,0,66


In [ ]:
#https://api.weather.gov/points/{latitude},{longitude}
#For example: https://api.weather.gov/points/39.7456,-97.0892

#my lat/lon - 364 Keno Rd, Burnside, KY  42519, United States
# DD (decimal degrees) - lat: 36.9426753  lon: -84.5725794
# DMS (degrees, minutes, seconds) - lat: N 36* 56' 33.63" lon: W 84* 34' 21.2844"
# https://api.weather.gov/points/39.7456,-97.0892

# https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast
# https://api.weather.gov/gridpoints/TOP/-97.36849969999999,39.849513999999985/forecast
# /zones/{type}/{zoneId}/forecast

url_2 = 'https://api.weather.gov/points/36.9426753,-84.5725794'
#print(url_2)
response = requests.get(url_2)
web_data = response.json()

web_data


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/39.8495,-97.3685',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-97.3685, 39.8495]},
 'properties': {'@id': 'https://api.weather.gov/p